# Raster Scan Simulation Example

## Setup

In [45]:
from datetime import datetime, timedelta
start_time = datetime.now()

## Radar system

In [46]:
from mpar_sim.radar import PhasedArrayRadar
from mpar_sim.beam.beam import RectangularBeam
from mpar_sim.look import RadarLook
# from mpar_sim.
from stonesoup.types.array import StateVector

radar = PhasedArrayRadar(
  position=StateVector([0, 0, 0]),
  position_mapping=(0, 2, 4),
  rotation_offset=StateVector([0, 0, 0]),
  # Array parameters
  n_elements_x=16,
  n_elements_y=16,
  element_spacing=0.5, # Wavelengths
  element_tx_power=10,
  # System parameters
  center_frequency=3e9,
  system_temperature=290,
  noise_figure=4,
  # Scan settings
  beam_shape=RectangularBeam,
  field_of_view=90,
  # Detection settings
  false_alarm_rate=1e-6,
)
radar.timestamp = start_time

Raster scan agent

In [47]:
from mpar_sim.agents.raster_scan import RasterScanAgent
import numpy as np

agent = RasterScanAgent(
    azimuth_scan_limits=np.array([-45, 45]),
    elevation_scan_limits=np.array([-1, 1]),
    azimuth_beam_spacing=0.85,
    elevation_beam_spacing=0.85,
    azimuth_beamwidth=15,
    elevation_beamwidth=15,
    bandwidth=100e6,
    pulsewidth=1e-6,
    prf=5e3,
    n_pulses=100,
)

## Tracker Components

Create tracker

In [48]:
from stonesoup.measures import Mahalanobis, Euclidean
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.updater.kalman import ExtendedKalmanUpdater
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.gater.distance import DistanceGater

# KF prediction model. Assuming it's matched to the true target model for now
transition_model = CombinedLinearGaussianTransitionModel([
    ConstantVelocity(1),
    ConstantVelocity(1),
    ConstantVelocity(0.0),
])
predictor = KalmanPredictor(transition_model)

updater = ExtendedKalmanUpdater(measurement_model=None)

hypothesizer = DistanceHypothesiser(
    predictor, updater, measure=Mahalanobis(), missed_distance=100)
gater = DistanceGater(hypothesizer, measure=Mahalanobis(), gate_threshold=15)


Create the data associator

In [49]:
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(gater)

Create the deleter

In [50]:
from stonesoup.deleter.time import UpdateTimeStepsDeleter, UpdateTimeDeleter
# deleter = UpdateTimeDeleter(timedelta(0.5))
deleter = UpdateTimeStepsDeleter(20)

Create the initiatoor

In [51]:
# from mpar_sim.
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator, SimpleMeasurementInitiator
import numpy as np
from mpar_sim.initiator.initators import MofNInitiator

initiator = MultiMeasurementInitiator(
    prior_state=GaussianState([0, 0, 0, 0, 0, 0], np.diag([0, 0, 0, 0, 0, 0])),
    measurement_model=None,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    min_points=2,
)

# initiator = MofNInitiator(
#     prior_state=GaussianState([0, 0, 0, 0, 0, 0], np.diag([0, 0, 0, 0, 0, 0])),
#     measurement_model=None,
#     deleter=deleter,
#     data_associator=data_associator,
#     updater=updater,
#     confirmation_threshold=[2,agent.n_positions+1],
# )

## Run the simulation

In [52]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.array import CovarianceMatrix
import random
# Set the simulation seed
# seed = 0
# np.random.seed(seed)
# random.seed(seed)

# Simulation-level parameters
n_steps = 100
include_noise = True

# Target generation parameters
initial_state_mean = StateVector([0, 0, 0, 0, 0, 0])
initial_state_covariance = CovarianceMatrix(np.diag([250, 10, 250, 10, 0, 0]))
initial_state = GaussianState(initial_state_mean, initial_state_covariance)
death_probability = 0.01
birth_probability = 0.8
target_rcs = 1


truths = set()
tracks = set()
all_truths = []
all_measurements = []
all_tracks = set()

time = start_time
for istep in range(n_steps):
  measurements = set()

  for truth in truths.copy():
    # Death to targets >:)
    if np.random.rand() <= death_probability:
      truths.remove(truth)

  # Targets, be reborn!
  for _ in range(np.random.poisson(birth_probability)):
    # Sample an initial state from the mean and covariance defined above
    state_vector = initial_state.state_vector + \
        initial_state.covar @ np.random.randn(initial_state.ndim, 1)
    state = GroundTruthState(
        state_vector=state_vector,
        timestamp=time,
    )
    
    # Give the target an RCS (constant for now)
    state.rcs = target_rcs
    # Add to the list of truths
    truth = GroundTruthPath([state])
    truths.add(truth)
    all_truths.append(truth)

  # TODO: Allocate resources/choose an action
  look = agent.act(current_time=time)
  look.tx_power = 100e3
  radar.load_look(look)
  # Simulate detections
  measurements |= radar.measure(truths, noise=include_noise)

  # Update tracks
  # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
  hypotheses = data_associator.associate(tracks,
                                         measurements,
                                         timestamp=time)
  associated_measurements = set()
  for track in tracks:
    hypothesis = hypotheses[track]
    if hypothesis.measurement:
      post = updater.update(hypothesis)
      track.append(post)
      associated_measurements.add(hypothesis.measurement)
    else:
      # When data associator says no detections are good enough, we'll keep the prediction
      track.append(hypothesis.prediction)

  # Carry out deletion and initiation
  tracks -= deleter.delete_tracks(tracks)
  tracks |= initiator.initiate(
      detections=measurements - associated_measurements,
      timestamp=time)
  

  # Advance simulation time
  # TODO: This gets more complicated when multiple tasks with different dwell times are scheduled
  dt = timedelta(seconds=look.dwell_time)
  time += dt

  # Advance target positions
  for truth in truths:
    truth.append(GroundTruthState(
        transition_model.function(truth[-1],
                                  noise=include_noise,
                                  time_interval=dt),
        timestamp=time))
    truth[-1].rcs = target_rcs
    
  # Save measurements and tracks for plotting
  # all_measurements.append(measurements)
  all_measurements.append(measurements)
  all_tracks |= tracks


## Plot simulation results

In [53]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_sensors(radar, "Radar")
plotter.plot_ground_truths(all_truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(all_tracks, [0,2])

plotter.fig